# <center>Estimating network structure from unreliable measurements</center>

In [262]:
import numpy as np
import networkx as nx
import random as random
from scipy.special import binom
from operator import itemgetter

## 1. Erdös-Rényi with independent edge measurements

### 1.1 Initialization

Number of nodes n and list of all node pairs.

In [345]:
n = 1000
node_pairs = list()
for i in range(n):
    for j in range(i,n):
        if i != j:
            node_pairs.append((i,j))

Graph parameters $\gamma$.

In [346]:
w = 1/n # prob of an edge between two nodes
print("w = {}".format(w))

w = 0.001


Generate graph.

In [347]:
G = nx.gnp_random_graph(n,w)

Data parameters $\theta$.

In [349]:
a = 0.8 # true positive rate
b = 0.1 # false positive rate

Generate data.

In [351]:
n_obs = 100 # nb of times we observe each node pair
N = {e:n_obs for e in node_pairs} # nb obs for each node pair
E = {e:0 for e in node_pairs} # nb edges seen during observations for each node pair

for k in range(n_obs):
    for e in node_pairs:
        dice = random.random()
        # if true positive
        if e in G.edges and dice < a:
            E[e] += 1
        # if false positive
        elif e not in G.edges and dice < b:
            E[e] += 1

### 1.2 Iterations

Choose number of repetitions of the whole algorithm and max iterations for each repetition.

In [352]:
repetitions = 1000
max_iter = 1000

Proceed.

In [ ]:
# save values at each big iter
A = list()
B = list()
W = list()


# we do multiple repetitions and then take the mean
for k in range(repetitions):
    
    try:
        
        # random initialization of the parameters
        w = random.random()
        a = random.random()
        b = random.random()

        # iter
        for l in range(max_iter):

            old_w, old_a, old_b = w, a, b

            # compute Qij
            Q = dict()
            for i,j in N:
                Q[i,j] = w * a**E[i,j] * (1-a)**(N[i,j]-E[i,j])
                Q[i,j] /= w * a**E[i,j] * (1-a)**(N[i,j]-E[i,j]) + (1-w) * b**E[i,j] * (1-b)**(N[i,j]-E[i,j])

            # update w,a,b
            w = sum(Q.values()) / binom(n,2)
            numerator_a, numerator_b = 0, 0
            denominator_a, denominator_b = 0, 0
            for i,j in N:
                numerator_a += Q[i,j] * E[i,j]
                numerator_b += (1-Q[i,j]) * E[i,j]
                denominator_a += Q[i,j] * N[i,j]
                denominator_b += (1-Q[i,j]) * N[i,j]
            a = numerator_a / denominator_a
            b = numerator_b / denominator_b

            # break if no sufficient evolution
            if np.abs(min([a-old_a, b-old_b, w-old_w])) < 0.001:
                break

        # save a,b,w
        A.append(a)
        B.append(b)
        W.append(w)
    
    except:
        continue
    
    
# print values
w_final = max([(w, W.count(w)/len(W)) for w in set(W)], key=itemgetter(1))
a_final = max([(a, A.count(a)/len(W)) for a in set(A)], key=itemgetter(1))
b_final = max([(b, B.count(b)/len(W)) for b in set(B)], key=itemgetter(1))
print("w = {}\na = {}\nb = {}\n".format(w_final, a_final, b_final))

Probability of the real graph.

In [258]:
qA = 1
for i,j in N:
    if (i,j) in G.edges:
        qA *= Q[i,j]
    else:
        qA *= 1 - Q[i,j]
print(qA)

1.0
